In [3]:
# --- Standard Library ---
import os
import re
import io
import sys
import time
import json
import math
import html
import random
import pathlib
import zipfile
import urllib.request
import urllib.parse
from collections import Counter, defaultdict

# --- Third-party libraries ---
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
import matplotlib.patches as mpatches
import networkx as nx
from bs4 import BeautifulSoup
from wordcloud import WordCloud

# --- NLP / Text ---
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords, words, wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

# --- NLTK Downloads ---
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('words')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!

True

In [10]:
# We will save the text about each performer as a txt file in the folder "performers"
save_dir = "managers"
os.makedirs(save_dir, exist_ok=True)

save_dir_career = "managers_career"
os.makedirs(save_dir_career, exist_ok=True)

# wiki api
baseurl = "https://en.wikipedia.org/w/api.php"
user_agent = "PremierLeagueManagerDownloader/1.0"

def get_category_members(category_title, depth=1):
    """
    Fetch all pages (and optionally subcategories) under a Wikipedia category.
    depth=1 means include one level of subcategories.
    """
    
    #Skip Premier League Manager...... and List of Premier League managers
    members = []
    cmcontinue = ""
    while True:
        params = {
            "action": "query",
            "list": "categorymembers",
            "cmtitle": f"Category:{category_title}",
            "cmlimit": "max",
            "format": "json"
        }
        if cmcontinue:
            params["cmcontinue"] = cmcontinue

        query = baseurl + "?" + urllib.parse.urlencode(params)
        req = urllib.request.Request(query, headers={"User-Agent": user_agent})
        with urllib.request.urlopen(req) as response:
            data = json.loads(response.read().decode("utf-8"))

        pages = data.get("query", {}).get("categorymembers", [])
        for p in pages:
            ns = p.get("ns")
            title = p.get("title")
            if ns == 0:
                members.append(title)
            elif ns == 14 and depth > 0:  # category namespace
                print(f"→ Exploring subcategory: {title}")
                members += get_category_members(title.replace("Category:", ""), depth=depth - 1)

        if "continue" in data:
            cmcontinue = data["continue"]["cmcontinue"]
            time.sleep(0.3)
        else:
            break
    return members

def fetch_wikitext(title):
    params = {
        "action": "query",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "titles": title,
        "format": "json"
    }
    query = baseurl + "?" + urllib.parse.urlencode(params)
    req = urllib.request.Request(query, headers={"User-Agent": user_agent})
    with urllib.request.urlopen(req) as response:
        data = json.loads(response.read().decode("utf-8"))

    pages = data["query"]["pages"]
    page = next(iter(pages.values()))
    try:
        return page["revisions"][0]["slots"]["main"]["*"]
    except (KeyError, IndexError):
        return None

In [5]:
# --- MAIN EXECUTION ---
category = "Premier League managers"
print(f"Fetching managers from category: {category}")
managers = get_category_members(category, depth=1)
print(f"Found {len(managers)} manager pages")

Fetching managers from category: Premier League managers
→ Exploring subcategory: Category:Lists of Premier League managers
Found 291 manager pages


In [9]:
import re

KEEP_SECTIONS = [
    "managerial career",
    "manager career",
    "coaching career",
    "coaching",
    "management",
    "managerial",
    "style of management",
    "style of play",
    "tactical",
    "tactics",
    "managerial statistics",
    "statistics",
]

REMOVE_SECTIONS = [
    "early life",
    "youth career",
    "playing career",
    "international career",
    "personal life",
    "media",
    "honours",
    "legacy",
    "post-retirement",
    "other work",
    "outside football",
]

def filter_manager_wikitext(raw):
    if not raw:
        return ""

    # Split into headers and content
    parts = re.split(r"(==[^=].*?==)", raw)

    cleaned = []

    i = 0
    while i < len(parts):
        chunk = parts[i]

        # Only process level-2 headers (==Header==)
        if re.match(r"^==[^=].*?==$", chunk.strip()):
            header = chunk.strip()
            header_name = re.sub(r"=+", "", header).strip().lower()

            # Skip remove sections
            if any(bad in header_name for bad in REMOVE_SECTIONS):
                i += 2
                continue

            # Keep section if it matches ANY keep keyword
            if any(good in header_name for good in KEEP_SECTIONS):
                # Take everything until the NEXT ==...== header
                section_text = []

                # Start reading the content after this header
                j = i + 1
                while j < len(parts):
                    # Stop if we hit another level-2 header
                    if re.match(r"^==[^=].*?==$", parts[j].strip()):
                        break
                    section_text.append(parts[j])
                    j += 1

                cleaned.append(header)
                cleaned.append("".join(section_text))

                i = j
                continue

        i += 1

    return "\n".join(cleaned).strip()


In [7]:
import re

def extract_section(raw, section_name):
    """
    Extract all text under == section_name == until the next ==...== header.
    Includes all === ... === subsections.
    """
    header_pattern = r"^==\s*([^=]+?)\s*==\s*$"
    lines = raw.split("\n")

    in_target = False
    collected = []

    for line in lines:
        m = re.match(header_pattern, line.strip())
        if m:
            current_header = m.group(1).strip().lower()

            # Start collecting
            if current_header == section_name.lower():
                in_target = True
                continue

            # If another section appears → stop
            if in_target:
                break

        if in_target:
            collected.append(line)

    return "\n".join(collected).strip()


In [12]:
for manager in managers:
    safe_title = manager.replace(" ", "_").replace("/", "_")
    wikitext = fetch_wikitext(manager)
    
    if not wikitext:
        print(f"Skipping {manager}, no text found")
        continue

    # Use the new KEEP/REMOVE + nested subsections filter
    filtered = filter_manager_wikitext(wikitext)

    filename = os.path.join(save_dir_career, f"{safe_title}.txt")
    with open(filename, "w", encoding="utf-8") as f:
        f.write(filtered)

    time.sleep(0.5)



KeyboardInterrupt: 

In [11]:
# saves the texts in the performers folder
for manager in managers:
    safe_title = manager.replace(" ", "_").replace("/", "_")
    wikitext = fetch_wikitext(manager)
    if not wikitext:
        print(f"Skipping {manager}, no text found")
        continue

    filename = os.path.join(save_dir, f"{safe_title}.txt")
    with open(filename, "w", encoding="utf-8") as f:
        f.write(wikitext)

    print(f"Saved {manager} → {filename}")
    time.sleep(0.5)  # be polite to Wikipedia

Saved Micky Adams → managers\Micky_Adams.txt
Saved Neil Adams (footballer) → managers\Neil_Adams_(footballer).txt
Saved Tony Adams → managers\Tony_Adams.txt
Saved Nigel Adkins → managers\Nigel_Adkins.txt
Saved Dick Advocaat → managers\Dick_Advocaat.txt
Saved Steve Agnew → managers\Steve_Agnew.txt
Saved Sam Allardyce → managers\Sam_Allardyce.txt
Saved Clive Allen → managers\Clive_Allen.txt
Saved Ruben Amorim → managers\Ruben_Amorim.txt
Saved Carlo Ancelotti → managers\Carlo_Ancelotti.txt


KeyboardInterrupt: 

In [ ]:
pages = [
    "Glossary of association football terms",
    "Association football tactics and skills",
    "Formation (association football)",
    "Association football positions",
    "Association football",
]

In [ ]:
# Folder to save pages
save_dir = "football_pages"
os.makedirs(save_dir, exist_ok=True)

# Wikipedia API setup
baseurl = "https://en.wikipedia.org/w/api.php"
user_agent = "FootballPageDownloader/1.0"
# 
def fetch_wikitext(title):
    """Fetch full Wikipedia page content (wikitext) for a given title."""
    params = {
        "action": "query",
        "prop": "revisions",
        "rvslots": "main",
        "rvprop": "content",
        "titles": title,
        "format": "json"
    }
    query = baseurl + "?" + urllib.parse.urlencode(params)
    req = urllib.request.Request(query, headers={"User-Agent": user_agent})
    with urllib.request.urlopen(req) as response:
        data = json.loads(response.read().decode("utf-8"))

    pages = data["query"]["pages"]
    page = next(iter(pages.values()))
    try:
        return page["revisions"][0]["slots"]["main"]["*"]
    except (KeyError, IndexError):
        return None

def save_txt(title, folder):
    """Fetch a Wikipedia page and save it as a text file in the given folder."""
    text = fetch_wikitext(title)
    if not text:
        print(f"⚠️ Skipped {title}: no text found.")
        return
    safe_name = title.replace("/", "_").replace(":", "_")
    path = os.path.join(folder, f"{safe_name}.txt")
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)
    print(f"✅ Saved {title}")

# Example: list of mentality / tactical / philosophy pages


In [ ]:
# Fetch and save all pages
for title in pages:
    save_txt(title, save_dir)
    time.sleep(0.5)  # be kind to the API

In [ ]:
save_dir = "football_terminology"
os.makedirs(save_dir, exist_ok=True)

baseurl = "https://en.wikipedia.org/w/api.php"
user_agent = "FootballTerminologyDownloader/1.0"

In [ ]:
# --- Fetch category pages ---
def get_category_members(category_title, depth=0):
    """Fetch all pages (and optionally subcategories) under a Wikipedia category."""
    members = []
    cmcontinue = ""
    while True:
        params = {
            "action": "query",
            "list": "categorymembers",
            "cmtitle": f"Category:{category_title}",
            "cmlimit": "max",
            "format": "json"
        }
        if cmcontinue:
            params["cmcontinue"] = cmcontinue

        query = baseurl + "?" + urllib.parse.urlencode(params)
        req = urllib.request.Request(query, headers={"User-Agent": user_agent})
        with urllib.request.urlopen(req) as response:
            data = json.loads(response.read().decode("utf-8"))

        pages = data.get("query", {}).get("categorymembers", [])
        for p in pages:
            ns = p.get("ns")
            title = p.get("title")
            if ns == 0:
                members.append(title)
            elif ns == 14 and depth > 0:
                print(f"→ Exploring subcategory: {title}")
                members += get_category_members(title.replace("Category:", ""), depth - 1)

        if "continue" in data:
            cmcontinue = data["continue"]["cmcontinue"]
            time.sleep(0.3)
        else:
            break
    return members


# --- Fetch full page text ---
def fetch_wikitext(title):
    params = {
        "action": "query",
        "prop": "revisions",
        "rvslots": "main",
        "rvprop": "content",
        "titles": title,
        "format": "json"
    }
    query = baseurl + "?" + urllib.parse.urlencode(params)
    req = urllib.request.Request(query, headers={"User-Agent": user_agent})
    with urllib.request.urlopen(req) as response:
        data = json.loads(response.read().decode("utf-8"))
    pages = data["query"]["pages"]
    page = next(iter(pages.values()))
    try:
        return page["revisions"][0]["slots"]["main"]["*"]
    except (KeyError, IndexError):
        return None


In [ ]:
category = "Association football terminology"
pages = get_category_members(category, depth=1)  # include subcategories too

print(f"Found {len(pages)} pages in category '{category}'.")

for title in pages:
    save_txt(title, save_dir)
    time.sleep(0.3)